In [1]:
import pandas as pd
import numpy as np

# 데이터 불러오기

In [2]:
hitter = pd.read_csv('선수데이터(전처리완료)/타자.csv') # 현재파일은 009단계까지 완료한 파일
pitcher = pd.read_csv('선수데이터(전처리완료)/투수.csv')
df = pd.read_csv('FA정보데이터/FA_정보.csv')
df

,선수명,생년월일,연도,이전팀,계약팀,계약기간,계약총액,계약평균액
0,오주원,1985-03-31,2020,키움,키움,2년,"70,000","35,000"
1,김태균,1982-05-29,2020,한화,한화,1년,"100,000","100,000"
2,오재원,1985-02-09,2020,두산,두산,3년,"190,000","63,333"
3,김태군,1989-12-30,2020,NC,NC,4년,"130,000","32,500"
4,이성열,1984-07-13,2020,한화,한화,2년,"140,000","70,000"
...,...,...,...,...,...,...,...,...
234,안경현,1970-02-13,2003,두산,두산,4년,"150,000","37,500"
235,양준혁,1969-05-26,2002,LG,삼성,4년,"272,000","68,000"
236,김민재,1973-01-03,2002,롯데,SK,4년,"100,000","25,000"
237,전준호,1969-02-15,2002,현대,현대,3년,"120,000","40,000"


# STATIZ 전처리

In [3]:
# 생년월일 오류 전처리 ("007.STATIZ에서 데뷔년도 크롤링 후 결측값 채우기"에서 조사한 내용)
# STATIZ와 KBO의 생년월일이 맞지 않는 몇몇을 판별 성공, 이들을 대체
df.loc[(df['선수명'] == '정종수') & (df['생년월일'] == '1980-06-08'), '생년월일'] = '1980-05-08'
df.loc[(df['선수명'] == '박가람') & (df['생년월일'] == '1986-11-09'), '생년월일'] = '1986-11-07'
df.loc[(df['선수명'] == '최현종') & (df['생년월일'] == '1985-08-05'), '생년월일'] = '1985-07-08'
df.loc[(df['선수명'] == '김태석') & (df['생년월일'] == '1967-02-12'), '생년월일'] = '1967-03-12'
df.loc[(df['선수명'] == '안병원') & (df['생년월일'] == '1973-10-04'), '생년월일'] = '1973-10-03'
df.loc[(df['선수명'] == '강관식') & (df['생년월일'] == '1984-04-12'), '생년월일'] = '1981-04-12'
df.loc[(df['선수명'] == '이정용') & (df['생년월일'] == '1997-03-26'), '생년월일'] = '1996-03-26'
df.loc[(df['선수명'] == '이호중') & (df['생년월일'] == '1992-10-13'), '생년월일'] = '1992-10-03'
df.loc[(df['선수명'] == '오동욱') & (df['생년월일'] == '2001-02-01'), '생년월일'] = '2001-02-04'

In [4]:
# 계약평균액이 결측치인 사람들은 FA에 제대로 계약이 안됐다고 판단하여 제거
df = df[df['계약평균액'].notnull()].reset_index(drop=True)

# 계약기간에서 '년'제거
df['계약기간'] = df['계약기간'].apply(lambda x: x.replace('년', '')).astype('int')
df

# 필요한 열만 가져오기
df = df[['선수명', '생년월일', '연도', '계약기간', '계약평균액']]
df

,선수명,생년월일,연도,계약기간,계약평균액
0,오주원,1985-03-31,2020,2,"35,000"
1,김태균,1982-05-29,2020,1,"100,000"
2,오재원,1985-02-09,2020,3,"63,333"
3,김태군,1989-12-30,2020,4,"32,500"
4,이성열,1984-07-13,2020,2,"70,000"
...,...,...,...,...,...
225,안경현,1970-02-13,2003,4,"37,500"
226,양준혁,1969-05-26,2002,4,"68,000"
227,김민재,1973-01-03,2002,4,"25,000"
228,전준호,1969-02-15,2002,3,"40,000"


# FA 계약평균액을 반영한 연봉수정

In [5]:
for name, birthday, year, contract_year, contract_money in zip(df['선수명'], df['생년월일'], df['연도'], df['계약기간'], df['계약평균액']):
    years = [year_element for year_element in range(year, year+contract_year)]
    
    # 타자 또는 투수에 연봉적용
    hitter.loc[(hitter['선수명'] == name) & (hitter['생년월일'] == birthday) & (hitter['연도'].isin(years)), '연봉'] = contract_money
    pitcher.loc[(pitcher['선수명'] == name) & (pitcher['생년월일'] == birthday) & (pitcher['연도'].isin(years)), '연봉'] = contract_money

# 해당 년도별 FA유무 표기

In [6]:
hitter['FA여부'] = 0
pitcher['FA여부'] = 0

for name, birthday, year in zip(df['선수명'], df['생년월일'], df['연도']):
    
    # 타자 또는 투수에 FA여부 적용
    hitter.loc[(hitter['선수명'] == name) & (hitter['생년월일'] == birthday) & (hitter['연도'] == year), 'FA여부'] = 1
    pitcher.loc[(pitcher['선수명'] == name) & (pitcher['생년월일'] == birthday) & (pitcher['연도'] == year), 'FA여부'] = 1

In [7]:
hitter.to_csv('선수데이터(전처리완료)/타자.csv', index=False, encoding='utf-8-sig')
pitcher.to_csv('선수데이터(전처리완료)/투수.csv', index=False, encoding='utf-8-sig')